In [1]:
import numpy as np

class SimplexMethod():
    def __init__(self, func_coefs, table):
        self.table = np.vstack((
            np.hstack((table, np.eye(table.shape[0]))),
            np.hstack((-func_coefs, np.zeros(table.shape[0])))
        ))
        
    def step(self):
        coefs = np.where(self.table[-1]<0, np.abs(self.table[-1]), -np.inf)
        q = np.argmax(coefs)
        B = self.table[:-1, 0]
        tmp = B/np.where(self.table[:-1, q]!=0, self.table[:-1, q], np.inf)
        tmp = np.where(tmp>0, tmp, np.inf)
        #print(tmp)
        p = np.argmin(tmp)
        #print(p)
        major_elem = self.table[p, q]
        #print(major_elem)
        new_tb = np.zeros_like(self.table)
        new_tb[p] = self.table[p]/major_elem
        for i in range(new_tb.shape[0]):
            if i == p:
                continue
            new_tb[i] = self.table[i]-self.table[i, q]*new_tb[p]
        self.table = new_tb
        
    def is_negative_coefs(self):
        if np.any(self.table[-1]<0):
            return True
    
    def get_table(self):
        return self.table
    
    def get_answer(self):
        x = np.zeros(self.table.shape[1]-1)
        for i, vec in enumerate(self.table.T[1:]):
            if vec[-1]==0:
                x[i] = self.table[np.where(vec==1)[0], 0]
        return x

In [2]:
table = np.array([
    [24, 2, 4],
    [40, 2, 8],
    [40, 4, 4]
], dtype=np.float64)

func_coefs = np.array([-24 , 0, 1], dtype=np.float64)
method = SimplexMethod(func_coefs, table)

In [3]:
method.get_table()

array([[24.,  2.,  4.,  1.,  0.,  0.],
       [40.,  2.,  8.,  0.,  1.,  0.],
       [40.,  4.,  4.,  0.,  0.,  1.],
       [24., -0., -1.,  0.,  0.,  0.]])

In [4]:
print('input data:')
for i in table:
    print('{} >= {}*x1 + {}*x2'.format(*i))
print('z = {} + {}*x1 + {}*x2 -> max'.format(*func_coefs))

input data:
24.0 >= 2.0*x1 + 4.0*x2
40.0 >= 2.0*x1 + 8.0*x2
40.0 >= 4.0*x1 + 4.0*x2
z = -24.0 + 0.0*x1 + 1.0*x2 -> max


In [5]:
while(method.is_negative_coefs()):
    method.step()
    print('step:')
    print(method.table)
    print()

step:
[[ 4.     1.     0.     1.    -0.5    0.   ]
 [ 5.     0.25   1.     0.     0.125  0.   ]
 [20.     3.     0.     0.    -0.5    1.   ]
 [29.     0.25   0.     0.     0.125  0.   ]]



In [6]:
tmp = method.get_table()
print('Output data:')
print(*tmp, sep='\n')
print('\ntarget func max: {}'.format(tmp[-1, 0]))
print('x = {}'.format(method.get_answer()))

Output data:
[ 4.   1.   0.   1.  -0.5  0. ]
[5.    0.25  1.    0.    0.125 0.   ]
[20.   3.   0.   0.  -0.5  1. ]
[29.     0.25   0.     0.     0.125  0.   ]

target func max: 29.0
x = [ 0.  5.  4.  0. 20.]
